In [ ]:
import boto3

In [ ]:
emr_client = boto3.client('emr')

In [ ]:
emr_client.terminate_job_flows?

In [ ]:
emr_client.list_clusters(ClusterStates=['STARTING', 'RUNNING', 'WAITING'])

In [ ]:
step_details = emr_client.add_job_flow_steps(
    JobFlowId='j-1YUL583YMCW3X',
    Steps=[
        {
            'Name': 'Compute Daily Revenue using Spark SQL',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': [
                    'spark-sql',
                    '-d',
                    's3.bucket=airetail',
                    '-f',
                    's3://airetail/scripts/08_compute_daily_product_revenue_args.sql'
                ]
            }
        }
    ]
)

In [ ]:
step_details['StepIds'][0]

In [ ]:
step_waiter = emr_client.get_waiter("step_complete")
step_waiter.wait(
    ClusterId='j-1YUL583YMCW3X',
    StepId=step_details['StepIds'][0],
    WaiterConfig={
        "Delay": 30,
        "MaxAttempts": 10
    }
)
emr_client.terminate_job_flows(JobFlowIds=['j-1YUL583YMCW3X'])

In [ ]:
emr_terminate_waiter = emr_client.get_waiter("cluster_terminated")

print('Waiting for Cluster to be Terminated')

emr_terminate_waiter.wait(
    ClusterId='j-1YUL583YMCW3X',
    WaiterConfig={
        "Delay": 30,
        "MaxAttempts": 10
    }
)

print('Cluster Terminated Completely')